<a href="https://colab.research.google.com/github/missLaiba22/Encoder-Decoder/blob/main/SP22_BAI_022_nlp_Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Laiba Idrees(SP22-BAI-022)

Aliza Yousaf(SP22-BAI-006)

Hafsa Atiq(SP22-BAI-016)

Fatima Zafar(SP22-BAI-014)

Shugufta Zahra(SP22-BAI-046)

Assignment 5

Ma'am Zobia Rehman

In [ ]:
!pip install tensorflow-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires tensorboard<2.18,>=2.17, but you have tensorboard 2.18.0 which is incompatible.


In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch


In [ ]:
# Load small subset of CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")
small_train_data = dataset["train"].select(range(500))
small_val_data = dataset["validation"].select(range(100))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    outputs = batch["highlights"]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_train_data = small_train_data.map(preprocess_data, batched=True)
tokenized_val_data = small_val_data.map(preprocess_data, batched=True)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Freeze encoder layers
for param in model.encoder.parameters():
    param.requires_grad = False

In [ ]:
# Training arguments with optimizations
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="no",  # Disable evaluation during training
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
)

# Train the model
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=62, training_loss=8.463310980027721, metrics={'train_runtime': 18.0901, 'train_samples_per_second': 27.639, 'train_steps_per_second': 3.427, 'total_flos': 33564766765056.0, 'train_loss': 8.463310980027721, 'epoch': 0.992})

In [ ]:
# Save fine-tuned model
model.save_pretrained("t5_fine_tuned")
tokenizer.save_pretrained("t5_fine_tuned")

('t5_fine_tuned/tokenizer_config.json',
 't5_fine_tuned/special_tokens_map.json',
 't5_fine_tuned/spiece.model',
 't5_fine_tuned/added_tokens.json')

In [ ]:
import shutil

# Zip the model directory
shutil.make_archive("t5_fine_tuned_model", 'zip', "t5_fine_tuned_model")


'/content/t5_fine_tuned_model.zip'

In [ ]:
from google.colab import files

# Download the zipped model
files.download("t5_fine_tuned_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Import libraries
import torch

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Example article for testing
article = """ISLAMABAD, RAWALPINDI: At least 11 people were killed after a bus met an accident and overturned on the M-14 Motorway near Fateh Jang on Monday.

With the deceased comprising six women and five men, the number of those wounded in the accident stands at 22, said the rescue officials.


The victims hailed from Bahawalpur, Vehari, Sharaqpur and Islamabad, added the rescue officials.

Furthermore, rescue officials said that six people, including three women, have been shifted to Benazir Bhutto Hospital, whereas one injured has been transferred to Islamabad and District Headquarter Hospital, each.

Play Video
Meanwhile, the Motorway Police spokesperson has said that the bus, travelling from Bahawalpur to Islamabad, met a deadly accident due to the driver's negligence.

Reacting to the incident, Punjab Chief Minister Maryam Nawaz has expressed grief and prayed for the speedy recovery of the injured."""

# Move input to the same device as the model
input_ids = tokenizer("summarize: " + article, return_tensors="pt").input_ids.to(device)

# Generate summary
summary_ids = model.generate(input_ids, max_length=64, num_beams=4, length_penalty=2.0, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:", summary)


Summary: the number of those wounded in the accident stands at 22. the bus met an accident and overturned on the M-14 motorway. the bus was travelling from Bahawalpur to Islamabad.
